# MQTT 

In this notebook we will use PAHO as our MQTT broker.

- https://pypi.org/project/paho-mqtt/
- 

## 0. Install packages

In [2]:
!pip install paho-mqtt

Processing c:\users\31653\appdata\local\pip\cache\wheels\78\ec\85\b657c03288b6e363c90369c96ee04eea9abb9be78cadb7855b\paho_mqtt-1.5.1-py3-none-any.whl


## 1. Steve's internet guide

In [2]:
# source: http://www.steves-internet-guide.com/python-mqtt-publish-subscribe/
# source: http://www.steves-internet-guide.com/mqtt-username-password-example/ => if you want to use eclipse you need account+pw

import time
import paho.mqtt.client as paho
#broker="broker.hivemq.com"
broker="mqtt.eclipse.org"
#define callback
def on_message(client, userdata, message):
    time.sleep(1)
    print("received message =",str(message.payload.decode("utf-8")))

client= paho.Client("client-001") #create client object client1.on_publish = on_publish #assign function to callback client1.connect(broker,port) #establish connection client1.publish("house/bulb1","on")
######Bind function to callback
client.on_message=on_message
#####
print("connecting to broker ",broker)
client.connect(broker)#connect
client.loop_start() #start loop to process received messages
print("subscribing ")
client.subscribe("house/bulb1")#subscribe
time.sleep(2)
print("publishing ")
client.publish("house/bulb1","on")#publish
time.sleep(4)
client.disconnect() #disconnect
client.loop_stop() #stop loop


connecting to broker  mqtt.eclipse.org


TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [9]:
#source: http://www.steves-internet-guide.com/into-mqtt-python-client/
import paho.mqtt.client as mqtt #import the client1
broker_address="mqtt.eclipseprojects.io" #use external broker
client = mqtt.Client("P1") #create new instance
client.connect(broker_address) #connect to broker
client.publish("house/main-light","OFF")#publish
print('done')

done


In [10]:
#source: http://www.steves-internet-guide.com/into-mqtt-python-client/
import paho.mqtt.client as mqtt #import the client1
broker_address="mqtt.eclipseprojects.io" #use external broker
print("creating new instance")
client = mqtt.Client("P1") #create new instance
print("connecting to broker")
client.connect(broker_address) #connect to broker
print("Subscribing to topic","house/bulbs/bulb1")
client.subscribe("house/bulbs/bulb1")
print("Publishing message to topic","house/bulbs/bulb1")
client.publish("house/bulbs/bulb1","OFF")

creating new instance
connecting to broker
Subscribing to topic house/bulbs/bulb1
Publishing message to topic house/bulbs/bulb1


## 2. Boek Krijn

In [31]:
#basis script publisher zie p.250
import paho.mqtt.client as mqtt

mqtt_topic = 'roomTemp'
mqtt_server_ip="mqtt.eclipseprojects.io" #use external broker

client = mqtt.Client()

client.connect(mqtt_server_ip, 1883) #use port 1883 for unsafe 
client.publish(mqtt_topic, '20 graden')
client.disconnect
print('done')

done


In [37]:
#basis script subscriber => create a separate python file (or notebook) and run it. Then run the previous cell.
import paho.mqtt.client as mqtt
mqtt_topic = 'roomTemp'
mqtt_server_ip="mqtt.eclipseprojects.io" #use external broker

def on_connect(client, userdata, flags, rc):
    print('verbonden')
    client.subscribe(mqtt_topic)
def on_message(client, userdata, msg):
    print(msg.topic +" "+ str(msg.payload))
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_server_ip, 1883, 60)
client.loop_forever()
client.disconnect

verbonden


KeyboardInterrupt: 

### 2b. Varianten op dit script

In [40]:
# as above but now with a while loop
import psutil
from time import sleep
import paho.mqtt.client as mqtt

mqtt_topic = 'roomTemp'
mqtt_server_ip="mqtt.eclipseprojects.io" #use external broker

client = mqtt.Client()

for i in range(0,3): # for loop that will run three times.
    cpu_use = psutil.cpu_percent()
    client.connect(mqtt_server_ip, 1883)
    #client.publish(mqtt_topic, "cpu use =")
    client.publish(mqtt_topic, cpu_use)
    client.disconnect
    print(cpu_use)
    sleep(5)
    
print('done')

6.7
11.9
4.6
done


In [39]:
# as in first script but now with json message string
import time
import paho.mqtt.client as mqtt
import json

mqtt_topic = 'roomTemp'
mqtt_server_ip = 'mqtt.eclipseprojects.io'

client = mqtt.Client()

msg_json = {
    "app_id": "test_project", "dev_id": "test_device",
    "payload_fields": {"temperature": 42, "co2": 42},
    "time": int(time.time() * 1e3)
}
msg_str = json.dumps(msg_json)

client.connect(mqtt_server_ip, 1883)
client.publish(mqtt_topic, msg_str)
client.disconnect()

0

### 2c. The script for the OpenEars metadata

In [43]:
#script that both reads cpu_temp and wifi_signal and publishes on topic 'open_ears_meta'. DOES NOT WORK ON WINDOWS ONLY LINUX
# This script will:
# 1. Import libaries
# 2. define and call functions to read wifi_signal and cpu_temp
# 3. set up mqtt and publish message
####libraries
import re
import time
from subprocess import check_output
import paho.mqtt.client as mqtt
import json

##### Define the functions
def read_signal():
    with open("/proc/net/wireless") as fp:
        lines = fp.readlines()
        for line in lines:
            match_obj = re.search(r'wlan0:\s*[^\s]*\s*[^\s]*\s*([^\s]*)', line)
            if match_obj:
                return(float(match_obj[1]))
    raise Exception("Unable to read /proc/net/wireless")

def get_temp():
    temp = check_output(['vcgencmd', 'measure_temp']).decode('UTF-8')
    return (findall('\d+\. \d+', temp)[0])

#call the functions
wifi_signal = read_signal()
cpu_temp= get_temp()

#set up mqtt
mqtt_broker = 'mqtt.eclipseprojects.io'
mqtt_topic = 'open_ears_meta'
#define a json message:
msg_json = {
    "device_id": "Michiel", 
    "payload_fields": {"cpu temp": cpu_temp, "wifi signal": wifi_signal},
    "time": int(time.time() * 1e3)
    }
message = json.dumps(msg_json)

client=mqtt.Client()
client.connect(mqtt_broker, 1883)
client.publish(mqtt_topic, message)
client.disconnect()
print(message)


FileNotFoundError: [Errno 2] No such file or directory: '/proc/net/wireless'

## 3. MQTT Sensemakers Amsterdam

In [16]:
# -*- coding: utf-8 -*-
"""Publish data to an MQTT topic."""

import paho.mqtt.publish as publish
import os
import json
import time

mqtt_host = "mqtt.sensemakersams.org"
mqtt_port = 31090
mqtt_user = "openears"
mqtt_password = "Op3nEar5"

msg_json = {
    # The message will be processed by the platform and stored in a database only if app_id corresponds to the MQTT username.
    "app_id": "openears",
    "dev_id": "OE001",
    "payload_fields": {"temperature": 42, "humidity": 42},
    # Epoch timestamp in milliseconds
    "time": int(time.time() * 1e3),
}
msg_str = json.dumps(msg_json)

auth = {"username": mqtt_user, "password": mqtt_password}
publish.single(
    "pipeline/openears/OE001", payload=msg_str, hostname=mqtt_host, port=mqtt_port, auth=auth
)

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [17]:
mosquitto_pub -t pipeline/openears/OE001 -h mqtt.sensemakersams.org -p 31090 -u openears -P Op3nEar5 -m '{"app_id": "openears", "dev_id": "OE001", "payload_fields": {"temperature": 42, "humidity": 42}}'

UsageError: Line magic function `%` not found.


In [9]:
# -*- coding: utf-8 -*-
"""Subscribe to an MQTT topic."""

import paho.mqtt.client as mqtt
import json
import os


def on_connect(mqttc, obj, flags, rc):
    print("Connected with result code " + str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    # Listen to all messages.
    mqttc.subscribe("pipeline/#", qos=0)
    # Listen to all messages from a project.
    # mqttc.subscribe("pipeline/test/+", qos=0)
    # Listen to all messages from a device within a project.
    # mqttc.subscribe("pipeline/test/test", qos=0)


def on_disconnect(mqttc, obj, rc):
    if rc != 0:
        print("Unexpected disconnection.")


def on_message(mqttc, obj, msg):
    print(
        "Received message '"
        + str(msg.payload)
        + "' on topic '"
        + msg.topic
        + "' with QoS "
        + str(msg.qos)
    )

    msg_json = json.loads(str(msg.payload.decode("utf-8")))
    print(json.dumps(msg_json, indent=4, sort_keys=True))


def on_publish(mqttc, obj, mid):
    print("Messaged ID: " + str(mid))


def on_subscribe(mqttc, obj, mid, granted_qos):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))


def on_log(mqttc, obj, level, string):
    print(level, string)


mqttc = mqtt.Client()

mqttc.on_message = on_message
mqttc.on_connect = on_connect
mqttc.on_publish = on_publish
mqttc.on_subscribe = on_subscribe

mqtt_host = "mqtt.sensemakersams.org"
mqtt_port = 31090
# mqtt_user = "public"
# mqtt_password = os.environ["PUBLIC_PASSWORD"]
mqtt_user = "test"
mqtt_password = os.environ["TEST_PASSWORD"]

mqttc.username_pw_set(mqtt_user, mqtt_password)
mqttc.connect(mqtt_host, mqtt_port, 60)

mqttc.loop_forever()

gaierror: [Errno 11002] getaddrinfo failed